In [13]:
import pandas as pd
import numpy as np

In [27]:
# data separation 
d = pd.read_csv("match_team_player_w_position.csv")

In [29]:
# data separation 
match_player = d.iloc[:, 106:]
#match_player.insert(0, "match_id", d.match_id)
#match_team.to_csv("match_team.csv", index = False)
match_player.to_csv("match_player.csv", index = False)

In [30]:
d.columns[106] # player starts from 106

'home_overall_rating_1'

In [31]:
# clean players
columns_to_drop = ['preferred', 'work_rate', "birthday", 'player_name',  'weight', 'height'] # delete ordinals
data = match_player
names = data.columns

def ifdrop(name, columns):
    for c in columns:
        if name.find(c) != -1:
            return True
    return False

to_drop = [x for x in names if ifdrop(x, columns_to_drop)]
print(len(to_drop)/22)
data.drop(columns = to_drop, inplace = True) # Now each player has 35 numerical attributes 
names = data.columns 

7.0


In [32]:
data.shape

(25979, 792)

In [163]:
data.columns[:72]

Index(['home_overall_rating_1', 'home_potential_1', 'home_crossing_1',
       'home_finishing_1', 'home_heading_accuracy_1', 'home_short_passing_1',
       'home_volleys_1', 'home_dribbling_1', 'home_curve_1',
       'home_free_kick_accuracy_1', 'home_long_passing_1',
       'home_ball_control_1', 'home_acceleration_1', 'home_sprint_speed_1',
       'home_agility_1', 'home_reactions_1', 'home_balance_1',
       'home_shot_power_1', 'home_jumping_1', 'home_stamina_1',
       'home_strength_1', 'home_long_shots_1', 'home_aggression_1',
       'home_interceptions_1', 'home_positioning_1', 'home_vision_1',
       'home_penalties_1', 'home_marking_1', 'home_standing_tackle_1',
       'home_sliding_tackle_1', 'home_gk_diving_1', 'home_gk_handling_1',
       'home_gk_kicking_1', 'home_gk_positioning_1', 'home_gk_reflexes_1',
       'home_position_1', 'home_overall_rating_2', 'home_potential_2',
       'home_crossing_2', 'home_finishing_2', 'home_heading_accuracy_2',
       'home_short_passing

In [170]:
data.columns[36*10:36*12]

Index(['home_overall_rating_11', 'home_potential_11', 'home_crossing_11',
       'home_finishing_11', 'home_heading_accuracy_11',
       'home_short_passing_11', 'home_volleys_11', 'home_dribbling_11',
       'home_curve_11', 'home_free_kick_accuracy_11', 'home_long_passing_11',
       'home_ball_control_11', 'home_acceleration_11', 'home_sprint_speed_11',
       'home_agility_11', 'home_reactions_11', 'home_balance_11',
       'home_shot_power_11', 'home_jumping_11', 'home_stamina_11',
       'home_strength_11', 'home_long_shots_11', 'home_aggression_11',
       'home_interceptions_11', 'home_positioning_11', 'home_vision_11',
       'home_penalties_11', 'home_marking_11', 'home_standing_tackle_11',
       'home_sliding_tackle_11', 'home_gk_diving_11', 'home_gk_handling_11',
       'home_gk_kicking_11', 'home_gk_positioning_11', 'home_gk_reflexes_11',
       'home_position_11', 'away_overall_rating_1', 'away_potential_1',
       'away_crossing_1', 'away_finishing_1', 'away_heading_acc

In [196]:
# sanity check
def positions(row):
    pos_home = [value for index, value in row.items() if index.find("home_position_") != -1]
    pos_away = [value for index, value in row.items() if index.find("away_position_") != -1]
    assert len(pos_home) == 11 and len(pos_away) == 11
    
    def counts(word, pos):
        return len([v for v in pos if v==word])
    
    row = [counts(loc, pos_home) for loc in ['forward', 'midfielder', 'defender', 'goal keeper']] + [counts(loc, pos_away) for loc in ['forward', 'midfielder', 'defender', 'goal keeper']]
    index =  ['home ' + loc for loc in ['forward', 'midfielder', 'defender', 'goal keeper']] + ['away ' + loc for loc in ['forward', 'midfielder', 'defender', 'goal keeper']]
    return pd.Series(row, index)  

In [239]:
index_obs = data.isnull().mean(axis=1) == 0
data_obs = data.loc[index_obs, :]
data_obs.shape

(18052, 792)

In [240]:
pos = data_obs.apply(positions, axis = 1)

In [241]:
def good_pos_keeper(x):
    return x[3] == 1 and x[7] == 1 and min(x) >= 1

In [242]:
good_pos = pos.apply(good_pos_keeper, axis = 1)
np.sum(good_pos)

15742

In [243]:
index_allpos = np.array(good_pos)
data_obs_allpos = data_obs.iloc[index_allpos, :]
data_obs_allpos.shape

(15742, 792)

In [132]:
row = data_obs_allpos.iloc[0,:]
row

home_overall_rating_1           67
home_potential_1                69
home_crossing_1                 29
home_finishing_1                36
home_heading_accuracy_1         25
                            ...   
away_gk_handling_11             23
away_gk_kicking_11              62
away_gk_positioning_11          23
away_gk_reflexes_11             23
away_position_11           forward
Name: 155, Length: 792, dtype: object

In [142]:
def transforms_three_one():
    t_gk = np.array(pd.read_csv("loading_gk.csv").iloc[:,1:])
    t_other = np.array(pd.read_csv("loading_other_three.csv").iloc[:,1:])
    transforms = {"forward":t_other.copy(), "midfielder":t_other.copy(), "defender":t_other.copy(), "goal keeper":t_gk}
    return transforms

In [143]:
t_gk = pd.read_csv("loading_gk.csv")
print(t_gk.shape)
t_other = pd.read_csv("loading_other_three.csv")
print(t_other.shape)
transforms = transforms_three_one()

(35, 5)
(35, 5)


In [144]:
def player_feature_by_loc(row, start, transforms, l = 36):
    values = row.values[start:(start+l)]
    loc = values[-1] 
    
    
    transform = transforms[loc] # np array
    assert transform.shape[0] == 35 and transform.shape[1] > 2 
    k = transform.shape[1] - 2
    s = transform[:,-1]
    m = transform[:,-2]
    features = (np.array(values[:-1]) - m) / s
    loading = transform[:,:-2]
    
    factors = features.dot(loading)
    
    newrow = list(factors)
    newrow.append(loc)
    
    rownames = [loc + ' factor ' + str(i+1) for i in range(k)]
    rownames.append('location')
    
    return pd.Series(newrow, index = rownames)

In [145]:
player_feature_by_loc(row, 0, transforms)

goal keeper factor 1        4.29835
goal keeper factor 2        2.48632
location                goal keeper
dtype: object

In [227]:
def row_feature_by_loc_side(row, transforms, home = True):
    starts = [i*36 for i in range(11)] if home else [i*36 for i in range(11, 22, 1)] 
    word = 'home_' if home else 'away_'
     
    players = [player_feature_by_loc(row, i, transforms, l=36) for i in starts]
    assert len(players) == 11
    newrow = []
    names = []
            
    # combine 
    for loc in ['forward', 'midfielder', 'defender']:
        pls = [p[:-1] for p in players if p[-1] == loc]
        assert len(pls) > 0
        k = pls[0].shape[0]
        pls = np.array(pls)
        
        #newrow.extend(pls.max(axis = 0))
        newrow.extend(pls.mean(axis = 0))
        #names.extend([word + loc + '_max_factor_' + str(i+1) for i in range(k)])
        names.extend([word + loc + '_mean_factor_' + str(i+1) for i in range(k)])
    
    pls = [p[:-1] for p in players if p[-1] == 'goal keeper']
    assert len(pls) == 1
    k = pls[0].shape[0]
    newrow.extend(np.array(pls[0]))
    names.extend([word + 'goalkeeper' + '_factor_' + str(i+1) for i in range(k)])
    
    return pd.Series(newrow, names)

def row_feature_by_loc(row, transforms):
    home = row_feature_by_loc_side(row, transforms, home = True)
    away = row_feature_by_loc_side(row, transforms, home = False)
    return pd.concat([home, away], axis = 0)

In [244]:
data_obs_allpos.shape

(15742, 792)

In [245]:
newrows = data_obs_allpos.apply(lambda row: row_feature_by_loc(row, transforms), axis = 1)

In [246]:
print(newrows.columns)
data_player_cleaned = newrows

Index(['home_forward_mean_factor_1', 'home_forward_mean_factor_2',
       'home_midfielder_mean_factor_1', 'home_midfielder_mean_factor_2',
       'home_defender_mean_factor_1', 'home_defender_mean_factor_2',
       'home_goalkeeper_factor_1', 'home_goalkeeper_factor_2',
       'away_forward_mean_factor_1', 'away_forward_mean_factor_2',
       'away_midfielder_mean_factor_1', 'away_midfielder_mean_factor_2',
       'away_defender_mean_factor_1', 'away_defender_mean_factor_2',
       'away_goalkeeper_factor_1', 'away_goalkeeper_factor_2'],
      dtype='object')


In [254]:
def transforms_four(file):
    t = np.array(pd.read_csv(file).iloc[:,1:])
    transforms = {"forward":t.copy(), "midfielder":t.copy(), "defender":t.copy(), "goal keeper":t.copy()}
    return transforms

In [257]:
transforms_v_4 = transforms_four("loading_all_four.csv")
data_player_cleaned_four = data_obs_allpos.apply(lambda row: row_feature_by_loc(row, transforms_v_4), axis = 1)

In [259]:
transforms_v_love = transforms_four("loading_all_four_LOVE.csv")
data_player_cleaned_love = data_obs_allpos.apply(lambda row: row_feature_by_loc(row, transforms_v_love), axis = 1)

In [247]:
# combine with team data
data_team = pd.read_csv("final_match_team.csv").drop(columns = ["season", "country_name", "league_name"])
data_team_keep = data_team.loc[index_obs,:].iloc[index_allpos, :]
print("data team shape", data_team_keep.shape)
data_all = pd.concat([data_team_keep.drop(columns = ["outcome"]), data_player_cleaned], axis=1)
print("data all shape", data_all.shape)

data team shape (15742, 29)
data all shape (15742, 44)


In [248]:
rows_complete = np.mean(np.isnan(np.array(data_all)), axis = 1) == 0

data_all = pd.concat([data_team_keep, data_player_cleaned], axis=1)
#data_later = data_all.iloc[later_times, :]
data_complete = data_all.iloc[rows_complete,:]
data_complete.shape

(14497, 45)

In [249]:
data_complete.to_csv("match_team_player_latent.csv")

In [258]:
data_complete_v_4 = pd.concat([data_team_keep, data_player_cleaned_four], axis=1).iloc[rows_complete,:]
data_complete_v_4.to_csv("match_team_player_latent_allfour.csv")

In [260]:
data_complete_v_love = pd.concat([data_team_keep, data_player_cleaned_love], axis=1).iloc[rows_complete,:]
data_complete_v_love.to_csv("match_team_player_latent_love.csv")

In [278]:
data_complete_v_love

,outcome,stage,match_id,home_team_goal,away_team_goal,home_buildUpPlaySpeed,home_buildUpPlayPassing,home_chanceCreationPassing,home_chanceCreationCrossing,home_chanceCreationShooting,...,away_midfielder_mean_factor_3,away_midfielder_mean_factor_4,away_defender_mean_factor_1,away_defender_mean_factor_2,away_defender_mean_factor_3,away_defender_mean_factor_4,away_goalkeeper_factor_1,away_goalkeeper_factor_2,away_goalkeeper_factor_3,away_goalkeeper_factor_4
454,H,29,665745,2,0,50.0,35.0,70.0,50.0,60.0,...,0.245656,0.322860,-0.735905,-0.741260,0.543201,0.194031,-2.004165,-1.997430,-1.245761,3.429107
456,D,29,665749,1,1,65.0,60.0,60.0,40.0,50.0,...,0.842579,0.361712,-0.693405,-1.379331,0.960039,0.297416,-1.349079,-1.479803,-1.058453,3.673546
471,H,30,665771,2,1,65.0,60.0,50.0,40.0,50.0,...,0.970922,0.447033,-0.916728,-0.455901,0.564429,0.350900,-1.796121,-1.458332,-1.665051,3.300425
517,H,1,838468,1,0,50.0,60.0,50.0,50.0,50.0,...,0.920976,0.388222,-0.436354,-0.510618,1.058906,0.256612,-1.075425,-1.128977,-0.805173,3.587556
524,H,10,838581,3,2,35.0,40.0,45.0,50.0,45.0,...,0.939894,-0.356446,-0.854751,-0.476093,1.154927,-0.564621,-1.190631,-0.836655,-1.244232,3.516666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25972,D,8,1992089,3,3,62.0,46.0,47.0,50.0,54.0,...,0.336355,-0.575314,-1.330043,-1.214379,0.286127,-0.468327,-2.674238,-1.750489,-1.898702,2.842223
25974,H,9,1992091,1,0,52.0,48.0,39.0,39.0,37.0,...,0.500540,-0.468546,-1.045740,-0.649071,0.355513,-0.384502,-2.924575,-2.289657,-2.409919,2.627540
25975,A,9,1992092,1,2,53.0,56.0,38.0,53.0,46.0,...,0.324686,-0.282623,-0.634127,-0.406264,0.391094,-0.491420,-2.732584,-1.845860,-2.066162,3.203530
25976,H,9,1992093,2,0,50.0,54.0,52.0,54.0,46.0,...,0.892170,-0.435830,-0.629947,-0.697023,0.976960,-0.142153,-2.517385,-2.505583,-1.985168,3.596715


# Now extract odds data to see if our prediction can make money

In [271]:
print(d.columns[31:63])
match_betting_odds = d.loc[index_obs,:].iloc[index_allpos, 32:62].iloc[rows_complete,:]
match_betting_odds.shape

Index(['away_player_11', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA',
       'BSH', 'BSD', 'BSA', 'country_name'],
      dtype='object')


(14497, 30)

In [272]:
def maximal_odds(row):
    newrow  = []
    for i in range(3):
        odds = row[np.arange(10) * 3 + i]
        if np.mean(np.isnan(odds)) == 1:
            newrow.append(np.nan)
        else:
            newrow.append(np.nanmax(odds))

    return pd.Series(newrow, index = ['max_odds_H', 'max_odds_D', 'max_odds_A'])

In [285]:
np.nanmin(match_betting_odds)

1.02

In [273]:
odds_rows = [maximal_odds(row) for _, row in match_betting_odds.iterrows()]
odds_rows = pd.concat(odds_rows, axis = 1)
odds_rows = odds_rows.transpose()
odds_rows.shape

(14497, 3)

In [280]:
odds_rows.index = data_complete.index
odds_rows

,max_odds_H,max_odds_D,max_odds_A
454,1.40,4.5,8.50
456,2.63,3.3,2.88
471,2.90,3.4,2.60
517,3.80,3.4,2.00
524,2.05,3.5,3.75
...,...,...,...
25972,NaN,NaN,NaN
25974,NaN,NaN,NaN
25975,NaN,NaN,NaN
25976,NaN,NaN,NaN


In [281]:
np.sum(odds_rows.isnull().mean(axis = 1) == 0)

13520

In [282]:
data_complete_love_with_odds = pd.concat([data_complete_v_love, odds_rows], axis = 1)
data_complete_love_with_odds.shape

(14497, 64)

In [283]:
data_complete_love_with_odds.to_csv("match_team_player_complete_withodds.csv", index = False)